In [1]:
import torch

### 可视化函数

In [4]:
import open3d as o3d

def visualize_samples(d_sample):
    print("Points in downsample set: ", len(d_sample))
    # d_sample = d_sample.numpy()
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### 数据准备

In [6]:
from SimAttention.dataloader import ModelNetDataSet

root = '/home/akira/下载/Pointnet2_PyTorch-master/byol_pcl/data/modelnet40_normal_resampled'
data = ModelNetDataSet(root)
DataLoader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True)

result = 0
for point in DataLoader:
    result, _ = point
    break
    
print(result.shape, type(result))
demo = result[0].reshape(-1, 3)
print(demo.shape)

The size of train data is 9843
torch.Size([4, 10000, 3]) <class 'torch.Tensor'>
torch.Size([10000, 3])


In [7]:
demo = demo.numpy()
visualize_samples(demo)

Points in downsample set:  10000


### FPS Method checked

In [36]:
def fps(xyz, npoint): 
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        result: sampled pointcloud , [B, npoint, 3]
    """
    
    device = xyz.device
    if xyz.dim() < 3:
        xyz = xyz.reshape((1, -1, 3))
    B, N, C = xyz.shape
    result = torch.zeros(B, npoint, C).to(device)
    
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)     
    distance = torch.ones(B, N).to(device) * 1e10                       

    batch_indices = torch.arange(B, dtype=torch.long).to(device)        
    
    barycenter = torch.sum((xyz), 1)                                    
    barycenter = barycenter/xyz.shape[1]
    barycenter = barycenter.view(B, 1, 3)

    dist = torch.sum((xyz - barycenter) ** 2, -1)
    farthest = torch.max(dist,1)[1]                                     

    for i in range(npoint):
        centroids[:, i] = farthest                                      
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)        
        dist = torch.sum((xyz - centroid) ** 2, -1)                     
        mask = dist < distance
        distance[mask] = dist[mask]                                     
        farthest = torch.max(distance, -1)[1]                           
    
    for b in range(0, B):
        for index in range(0, npoint):
            # todo 后面是不是可以改成生成式的形式？
            result[b, index] = xyz[b, centroids[b, index]]
    
    # r = [xyz[b, centroids[b, i]].item() for b in range(0, B) for i in range(0, npoint)]
    
    return result

In [29]:
result.shape

torch.Size([4, 10000, 3])

In [30]:
o_fps = fps(result, 1024)
visualize_samples(o_fps[0].reshape(-1, 3))

Points in downsample set:  1024


In [31]:
o_fps.shape

torch.Size([4, 1024, 3])

### Slice Method checked

In [56]:
def get_slice(point_set, xyz_dim, index_slice, npoints):
    device = point_set.device
    B, _, C = point_set.shape
    result = torch.zeros((B, npoints, C)).to(device)
    
    def get_slice_index(index_slice, ratio_per_slice, overlap_ratio, num_all_points):
        start_index = index_slice * (ratio_per_slice - overlap_ratio) * num_all_points
        end_index = start_index + ratio_per_slice * num_all_points
        return int(start_index), int(end_index)

    def get_1_slice(point_set, xyz_dim, index_slice, npoints):
        # xyz_dim: 0, 1, 2 for x, y, z
        print('point_set shape: ', point_set.shape)
        start_index, end_index = get_slice_index(index_slice, 0.4, 0.1, len(point_set))
        patch_index = torch.argsort(point_set, dim=0)[start_index: end_index, xyz_dim]
        patch = point_set[patch_index]
        
        if len(patch_index) > npoints:
            patch = fps(patch, npoints)
        return patch

    for b in range(0, B):
        result[b] = get_1_slice(point_set[b], xyz_dim, index_slice, npoints)
    return result

In [57]:
o_slice = get_slice(result, 1, 1, 1024)
o_slice.shape

point_set shape:  torch.Size([10000, 3])
point_set shape:  torch.Size([10000, 3])
point_set shape:  torch.Size([10000, 3])
point_set shape:  torch.Size([10000, 3])


torch.Size([4, 1024, 3])

In [38]:
o_slice_1 = o_slice[0]
o_slice_1.shape

torch.Size([1024, 3])

In [72]:
visualize_samples(o_slice[3])

Points in downsample set:  1024


### Cube Method checked

In [40]:
def get_cube(point_set, side_length, npoints):
    device = point_set.device
    B, _, C = point_set.shape
    result = torch.zeros((B, npoints, C)).to(device)

    def point_in_cube(point_xyz, side_length):
        flag = True
        for i in range(0, len(point_xyz)):
            if abs(point_xyz[i]) >= (side_length / 2):
                flag = False
                break
        return flag

    def get_1_cube(point_set, side_length, npoints):
        sample_index = []
        for i in range(0, len(point_set)):
            if point_in_cube(point_set[i], side_length):
                sample_index.append(i)
        if len(sample_index) >= npoints:
            r = fps(point_set[sample_index], npoints)
            return r
        else:
            return get_1_cube(point_set, side_length + 0.1, npoints)

    for i in range(point_set.shape[0]):
        result[i] = get_1_cube(point_set[i], side_length, npoints)
    return result

In [41]:
o_cube = get_cube(result, 0.1, 1024)
o_cube.shape

torch.Size([4, 1024, 3])

In [42]:
o_cube_1 = o_cube[0]
o_cube_1.shape

torch.Size([1024, 3])

In [69]:
visualize_samples(o_cube[3])

Points in downsample set:  1024


### Sphere Method checked

In [61]:
def get_sphere(point_set, radius, npoints):
    device = point_set.device
    B, _, C = point_set.shape
    result = torch.zeros((B, npoints, C)).to(device)

    def point_in_ball(point_xyz, center_xyz, radius):
        flag = False
        dist = 0
        for i in range(3):
            dist += (point_xyz[i] - center_xyz[i]) ** 2
        if dist <= radius ** 2:
            flag = True
        return flag

    def get_1_sphere(point_set, radius, npoints):
        center_xyz = torch.zeros([3]).to(device)
        sample_index = []
        for i in range(0, len(point_set)):
            if point_in_ball(point_set[i], center_xyz, radius):
                sample_index.append(i)
        if len(sample_index) >= npoints:
            r = fps(point_set[sample_index], npoints)
            return r
        else:
            return get_1_sphere(point_set, radius + 0.1, npoints)

    for i in range(point_set.shape[0]):
        result[i] = get_1_sphere(point_set[i], radius, npoints)
    return result

In [62]:
o_sphere = get_sphere(result, 0.1, 1024)
o_sphere.shape

torch.Size([4, 1024, 3])

In [66]:
visualize_samples(o_sphere[3])

Points in downsample set:  1024


### 测试PCT的函数

In [73]:
import numpy as np

def random_point_dropout(batch_pc, max_dropout_ratio=0.875):
    device = batch_pc.device
    # batch_pc: BxNx3
    for b in range(batch_pc.shape[0]):
        dropout_ratio = np.random.random()*max_dropout_ratio  # 0~0.875
        drop_idx = np.where(np.random.random((batch_pc.shape[1])) <= dropout_ratio)[0]
        drop_idx = torch.from_numpy(drop_idx).to(device)
        r = batch_pc.clone()
        if len(drop_idx) > 0:
            r[b, drop_idx, :] = batch_pc[b, 0, :]  # set to the first point
    return r

In [74]:
o_rpd = random_point_dropout(result)
o_rpd.shape

torch.Size([4, 10000, 3])

In [75]:
def random_scale_point_cloud(batch_data, scale_low=0.8, scale_high=1.25):
    """ Randomly scale the point cloud. Scale is per point cloud.
        Input:
            BxNx3 array, original batch of point clouds
        Return:
            BxNx3 array, scaled batch of point clouds
    """
    B, N, C = batch_data.shape
    device = batch_data.device
    scales = np.random.uniform(scale_low, scale_high, B)
    scales = torch.from_numpy(scales).to(device)
    for batch_index in range(B):
        batch_data[batch_index, :, :] *= scales[batch_index]
    return batch_data

In [77]:
o_rspc = random_scale_point_cloud(result)
o_rspc.shape

torch.Size([4, 10000, 3])

In [78]:
def shift_point_cloud(batch_data, shift_range=0.1):
    """ Randomly shift point cloud. Shift is per point cloud.
        Input:
          BxNx3 array, original batch of point clouds
        Return:
          BxNx3 array, shifted batch of point clouds
    """
    B, N, C = batch_data.shape
    device = batch_data.device
    shifts = np.random.uniform(-shift_range, shift_range, (B, 3))
    shifts = torch.from_numpy(shifts).to(device)
    for batch_index in range(B):
        batch_data[batch_index, :, :] += shifts[batch_index, :]
    return batch_data

In [79]:
o_spc = shift_point_cloud(result)
o_spc.shape

torch.Size([4, 10000, 3])

### PointWOLF 部分

In [90]:
class Batch_PointWOLF(object):
    def __init__(self):
        self.num_anchor = 4
        self.sample_type = 'fps'
        self.sigma = 0.5

        self.R_range = (-abs(10), abs(10))
        self.S_range = (1., 3)
        self.T_range = (-abs(0.25), abs(0.25))

    def __call__(self, b_pos):
        device = b_pos.device
        b_pos = b_pos.cpu().numpy()
        M = self.num_anchor  # (Mx3)
        B, N, C = b_pos.shape  # (N)
        result_aug = np.zeros((B, N, C))

        for i in range(0, B):
            pos = b_pos[i]
            if self.sample_type == 'random':
                idx = np.random.choice(N, M)  # (M)
            elif self.sample_type == 'fps':
                idx = self.fps(pos, M)  # (M)
            pos_anchor = pos[idx]  # (M,3), anchor point

            pos_repeat = np.expand_dims(pos, 0).repeat(M, axis=0)  # (M,N,3)
            pos_normalize = np.zeros_like(pos_repeat, dtype=pos.dtype)  # (M,N,3)
            pos_normalize = pos_repeat - pos_anchor.reshape(M, -1, 3)

            # Local transformation at anchor point
            pos_transformed = self.local_transformaton(pos_normalize)  # (M,N,3)

            # Move to origin space
            pos_transformed = pos_transformed + pos_anchor.reshape(M, -1, 3)  # (M,N,3)

            pos_new = self.kernel_regression(pos, pos_anchor, pos_transformed)
            pos_new = self.normalize(pos_new)

            result_aug[i] = pos_new.astype('float32')

        return torch.from_numpy(result_aug).to(device)

    def kernel_regression(self, pos, pos_anchor, pos_transformed):
        M, N, _ = pos_transformed.shape

        # Distance between anchor points & entire points
        sub = np.expand_dims(pos_anchor, 1).repeat(N, axis=1) - np.expand_dims(pos, 0).repeat(M, axis=0)  # (M,N,3), d

        project_axis = self.get_random_axis(1)

        projection = np.expand_dims(project_axis, axis=1) * np.eye(3)  # (1,3,3)

        # Project distance
        sub = sub @ projection  # (M,N,3)
        sub = np.sqrt(((sub) ** 2).sum(2))  # (M,N)

        # Kernel regression
        weight = np.exp(-0.5 * (sub ** 2) / (self.sigma ** 2))  # (M,N)
        pos_new = (np.expand_dims(weight, 2).repeat(3, axis=-1) * pos_transformed).sum(0)  # (N,3)
        pos_new = (pos_new / weight.sum(0, keepdims=True).T)  # normalize by weight
        return pos_new

    def fps(self, pos, npoint):
        N, _ = pos.shape
        centroids = np.zeros(npoint, dtype=np.int_)  # (M)
        distance = np.ones(N, dtype=np.float64) * 1e10  # (N)
        farthest = np.random.randint(0, N, (1,), dtype=np.int_)
        for i in range(npoint):
            centroids[i] = farthest
            centroid = pos[farthest, :]
            dist = ((pos - centroid) ** 2).sum(-1)
            mask = dist < distance
            distance[mask] = dist[mask]
            farthest = distance.argmax()
        return centroids

    def local_transformaton(self, pos_normalize):
        M, N, _ = pos_normalize.shape
        transformation_dropout = np.random.binomial(1, 0.5, (M, 3))  # (M,3)
        transformation_axis = self.get_random_axis(M)  # (M,3)

        degree = np.pi * np.random.uniform(*self.R_range, size=(M, 3)) / 180.0 * transformation_dropout[:,
                                                                                 0:1]  # (M,3), sampling from (-R_range, R_range)

        scale = np.random.uniform(*self.S_range, size=(M, 3)) * transformation_dropout[:,
                                                                1:2]  # (M,3), sampling from (1, S_range)
        scale = scale * transformation_axis
        scale = scale + 1 * (scale == 0)  # Scaling factor must be larger than 1

        trl = np.random.uniform(*self.T_range, size=(M, 3)) * transformation_dropout[:,
                                                              2:3]  # (M,3), sampling from (1, T_range)
        trl = trl * transformation_axis

        # Scaling Matrix
        S = np.expand_dims(scale, axis=1) * np.eye(3)  # scailing factor to diagonal matrix (M,3) -> (M,3,3)
        # Rotation Matrix
        sin = np.sin(degree)
        cos = np.cos(degree)
        sx, sy, sz = sin[:, 0], sin[:, 1], sin[:, 2]
        cx, cy, cz = cos[:, 0], cos[:, 1], cos[:, 2]
        R = np.stack([cz * cy, cz * sy * sx - sz * cx, cz * sy * cx + sz * sx,
                      sz * cy, sz * sy * sx + cz * cy, sz * sy * cx - cz * sx,
                      -sy, cy * sx, cy * cx], axis=1).reshape(M, 3, 3)

        pos_normalize = pos_normalize @ R @ S + trl.reshape(M, 1, 3)
        return pos_normalize

    def get_random_axis(self, n_axis):
        axis = np.random.randint(1, 8, (
            n_axis))  # 1(001):z, 2(010):y, 3(011):yz, 4(100):x, 5(101):xz, 6(110):xy, 7(111):xyz
        m = 3
        axis = ((axis[:, None] & (1 << np.arange(m))) > 0).astype(int)
        return axis

    def normalize(self, pos):
        pos = pos - pos.mean(axis=-2, keepdims=True)
        scale = (1 / np.sqrt((pos ** 2).sum(1)).max()) * 0.999999
        pos = scale * pos
        return pos


In [91]:
aug_func = Batch_PointWOLF()
aug = aug_func(result)
aug.shape

torch.Size([4, 10000, 3])

### 新的model